In [21]:
from dotenv import load_dotenv
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import OpenAI  # 或其他LLM
from langchain.memory import ConversationBufferWindowMemory
from typing import List, Dict, Any
import json

from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
import os


# 1. 定义商品获取工具
class ProductSearchTool:
    """模拟商品搜索工具"""

    @tool
    def search_products(self, query: str) -> List[Dict[str, Any]]:
        """
        模拟根据查询条件获取商品列表
        实际应用中这里可以连接数据库或调用API
        """
        # 模拟商品数据库
        product_db = [
            {"id": 1, "name": "无线蓝牙耳机", "price": 299, "category": "电子产品", "brand": "SoundMaster"},
            {"id": 2, "name": "智能手表", "price": 999, "category": "电子产品", "brand": "TechGiant"},
            {"id": 3, "name": "有机棉T恤", "price": 89, "category": "服装", "brand": "EcoWear"},
            {"id": 4, "name": "不锈钢保温杯", "price": 129, "category": "生活用品", "brand": "KeepHot"},
            {"id": 5, "name": "便携式充电宝", "price": 159, "category": "电子产品", "brand": "PowerUp"},
        ]

        # 简单模拟搜索逻辑 - 实际应用中可以更复杂
        query = query.lower()
        results = []
        for product in product_db:
            if (query in product["name"].lower() or
                    query in product["category"].lower() or
                    query in product["brand"].lower()):
                results.append(product)

        return results[:3]  # 返回最多3个结果

# 2. 创建工具实例
product_tool = ProductSearchTool()

tools = [
    Tool(
        name="Product Search",
        func=product_tool.search_products,
        description="""用于搜索商品。输入应该是用户的商品需求描述，
        如'蓝牙耳机'或'智能家居设备'。输出是匹配的商品列表。"""
    )
]

# 3. 定义提示词模板
template = """你是一个智能商品推荐助手，帮助用户找到他们需要的商品。
你可以访问商品数据库来查找匹配的商品。

用户当前需求: {input}

你应该:
1. 分析用户需求，理解他们想要什么类型的商品
2. 使用工具查找匹配的商品
3. 向用户推荐最合适的1-3个商品，说明推荐理由
4. 如果找不到匹配商品，请明确输出提示信息，礼貌地告知用户并尝试询问更详细的需求
5. 如果你有调用工具进行查询，请在回应中显式地指出

请开始完成任务:"""

prompt = PromptTemplate(
    input_variables=["input"],
    template=template
)

# 4. 设置记忆和LLM
# memory = ConversationBufferWindowMemory(k=3)  # 记住最近3轮对话
# llm = OpenAI(temperature=0.3)  # 控制创造性，商品推荐需要准确性

# 5. 创建LLM链
# llm_chain = LLMChain(llm=llm, prompt=prompt)

# 6. 定义自定义输出解析器
class ProductOutputParser:
    def parse(self, text: str):
        # 这里可以添加更复杂的解析逻辑
        return text.strip()

output_parser = ProductOutputParser()

# 7. 智能体创建
load_dotenv()
llm = ChatOpenAI(
    model="deepseek-ai/DeepSeek-R1",
    temperature=0,
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_API_BASE")
)
llm_with_tools = llm.bind_tools(tools=tools)
llm = prompt|llm_with_tools|StrOutputParser()

# 9. 测试对话
def run_conversation():
    print("商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)")
    while True:
        user_input = input("用户: ")
        if user_input.lower() == '退出':
            print("商品推荐助手: 感谢使用，再见！")
            break

        for res in llm.stream({"input": user_input}):
            print(res)
        # response = llm.invoke(input=user_input,stream=True)
        # response = llm.stream({"input":user_input})
        # print(f"tool_calls: {response.tool_calls}")
        # print(f"商品推荐助手: {response}")

        break
if __name__ == "__main__":
    run_conversation()

商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)



























































































































































































































































































































































好的
，
我将
根据
您的
需求
“
蓝牙
耳机
”
进行
商品
搜索
和
推荐
。
现在
调用
工具
进行
查询
。










